In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import sys
import pandas as pd
sys.path.append('auto_text_classifier')
from atc.models.aml import AML
from atc.utils.data_utils import load_df
from atc.utils.metrics_utils import get_multi_class_report
from atc.utils.metrics_utils import get_model_metrics
from tqdm import tqdm_notebook
import numpy as np
from atc.models.aml import AML
from atc.utils.data_utils import load_df,get_data_report
from utils.data_split import DataGet,add_diff_content
import pickle
from tqdm import tqdm_notebook

Using TensorFlow backend.


## 预测多个模型结果

In [3]:
df_wandb = pd.read_csv('data/after_preprocess/task1_cv_large.csv')
df_wandb = df_wandb[df_wandb['State']=='finished']
df_wandb = df_wandb.sort_values("AUC_Value_dev",ascending=False)

In [6]:
df_wandb.head(2)

,Name,Agent,State,Notes,User,Tags,Created,Runtime,Sweep,batch_size,...,Recall_final_dev,Recall_test,acc_final_dev,acc_test,model_final_dev,model_save_dir,score_final_dev,score_test,tradeoff_final_dev,tradeoff_test
19,gentle-sweep-145,lt9lhzb6,finished,-,tabchen,NaN,2022-09-15T01:25:40.000Z,2477,31xks6k8,32,...,0.5152,0.0000,0.816952,0.842092,pred,model/task1/cv5/358a8e6d-3eaf-4df1-be4e-fedba3...,1.255452,1.269692,0.30,0.30
96,curious-sweep-68,lt9lhzb6,finished,-,tabchen,NaN,2022-09-14T05:27:29.000Z,2621,31xks6k8,32,...,0.5923,0.0228,0.793273,0.812182,pred,model/task1/cv5/1be9712d-10c8-4b72-9835-fc71ff...,1.241173,1.266382,0.22,0.22


In [4]:
df_wandb.head()[['AUC_Value_dev']]

,AUC_Value_dev
19,0.8081
96,0.8068
151,0.8063
154,0.8052
127,0.8051


In [13]:
up_num = 5
down_num = 5
kf_i = 0
dataset = DataGet("data/after_preprocess/task1_charlie_train.csv",n_splits=5)
df_train, df_dev, df_test = dataset.get_data(
        random_state=0, kf_i=kf_i, up_num=up_num, down_num=down_num)

In [55]:
df_final_dev = pd.read_csv('data/after_preprocess/task1_charlie_dev.csv')
df_final_test = pd.read_csv('data/after_preprocess/task1_charlie_test.csv')
df_final_test['Speaker'] = df_final_test['Speaker'].fillna("Other")
df_final_test['label'] = 0

In [56]:
df_dev.shape,df_test.shape,df_final_dev.shape,df_final_test.shape

((1309, 12), (3258, 12), (2811, 8), (4122, 8))

In [57]:
df_final_dev

,Id,Dialogue_id,Utterance_id,Sentence,Speaker,Label,fname,Dialogue_id_old
0,0,Charlie_S01E08.csv_1,0,"Well, Charlie, today is a very special day.",Teddy,0.0,Charlie_S01E08.csv,1
1,1,Charlie_S01E08.csv_1,1,It's your first birthday! The whole family's e...,Teddy,0.0,Charlie_S01E08.csv,1
2,2,Charlie_S01E08.csv_1,2,"Happy Birthday, sweetie.",Amy,0.0,Charlie_S01E08.csv,1
3,3,Charlie_S01E08.csv_1,3,"I love you, baby girl.",Bob,0.0,Charlie_S01E08.csv,1
4,4,Charlie_S01E08.csv_1,4,You're the best.,PJ,0.0,Charlie_S01E08.csv,1
...,...,...,...,...,...,...,...,...
2806,301,Charlie_S02E05.csv_17,0,"So elegant, so poised.",Others,0.0,Charlie_S02E05.csv,17
2807,302,Charlie_S02E05.csv_17,1,Such sophistication.,Others,0.0,Charlie_S02E05.csv,17
2808,303,Charlie_S02E05.csv_17,2,"Our finest pupil ever, Mrs.Krump.",Others,0.0,Charlie_S02E05.csv,17
2809,304,Charlie_S02E05.csv_17,3,Uh-oh.,Charlie,0.0,Charlie_S02E05.csv,17


In [65]:
df_list = [df_dev,df_test,df_final_dev,df_final_test]

In [66]:
after_process_dir = 'model/task1/final_merge/'

In [ ]:
for _,row in tqdm_notebook(df_wandb[:2].iterrows()):
    up_num = row['up_num']
    down_num = row['down_num']
    df_list = [add_diff_content(df,up_num,down_num) for df in df_list]
    model_name = row['model_name']
    
    ai = AML(save_dir=row['model_save_dir'].replace(model_name,""), config={"max_len": 256, "batch_size": 64})
    df_list = ai.get_list_result(
        df_list, model_list=[model_name])
    with open(os.path.join(after_process_dir,f"{row['Name']}.pkl"),'wb') as f:
        pickle.dump(df_list,f)

0it [00:00, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

config is :{'model_dir': '/root/autodl-nas/tabchen/auto_text_classifier/atc/configs/../data/en_deberta_v3_large', 'save_dir': 'model/task1/cv5/358a8e6d-3eaf-4df1-be4e-fedba3e6d6bc/en_deberta_v3_large', 'max_len': 256, 'batch_size': 64}
Training use origin loss ~~
Load model
Load finish



100%|██████████| 1/1 [02:16<00:00, 136.21s/it]

Release model


...

config is :{'model_dir': '/root/autodl-nas/tabchen/auto_text_classifier/atc/configs/../data/en_deberta_v3_large', 'save_dir': 'model/task1/cv5/bc8b7562-33c3-4c08-b6da-1d250f797a64/en_deberta_v3_large', 'max_len': 256, 'batch_size': 64}
Training use origin loss ~~
Load model
Load finish



100%|██████████| 1/1 [02:16<00:00, 136.47s/it]

Release model




  0%|          | 0/1 [00:00<?, ?it/s]

config is :{'model_dir': '/root/autodl-nas/tabchen/auto_text_classifier/atc/configs/../data/en_deberta_large', 'save_dir': 'model/task1/cv5/b29a0a63-906a-409a-af50-80563fd5e129/en_deberta_large', 'max_len': 256, 'batch_size': 64}
Training use origin loss ~~
Load model
Load finish


## 汇总最终结果

In [3]:
df_wandb = pd.read_csv('data/after_preprocess/task1_cv_large.csv')
df_wandb = df_wandb[df_wandb['State']=='finished']
df_wandb = df_wandb.sort_values("AUC_Value_dev",ascending=False)

In [4]:
from utils.metrics import *

Using TensorFlow backend.
2022-09-25 09:50:56.202557: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-09-25 09:50:56.232938: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2000000000 Hz
2022-09-25 09:50:56.244839: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4b74000b20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-25 09:50:56.244876: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-09-25 09:50:56.248668: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-09-25 09:50:56.262808: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:52:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCo

In [5]:
after_process_dir = 'model/task1/final_merge/'

In [6]:
def get_new_df(num_models):
    name_list = []
    new_df_list = []
    for _,row in tqdm_notebook(df_wandb[:num_models].iterrows()):
        name = row['Name']
        name_list.append(name)
        df_list = pickle.load(open(os.path.join(after_process_dir,f"{row['Name']}.pkl"),'rb'))
        for i,df in enumerate(df_list):
            if 'Label' in df.columns:
                df['label'] = df['Label']
            df[name] = df[row['model_name']]
            if len(new_df_list)<len(df_list):
                new_df_list.append(df)
            else:
                new_df_list[i][name] = df[name]
    return new_df_list,name_list

In [7]:
def get_scores(new_df_list,name_list):
    df_score_list = []
    for df in new_df_list:
        df['merge_mean'] = df[name_list].mean(axis=1)
        df_merge_score = search_model(df, model_list=['merge_mean'])
        df_score_list.append(df_merge_score)
    tradeoff = df_score_list[0].drop_duplicates("model").iloc[0]['tradeoff']
    test_score = df_score_list[1][df_score_list[1]['tradeoff']==tradeoff].iloc[0]['score']
    final_dev_score = df_score_list[2][df_score_list[2]['tradeoff']==tradeoff].iloc[0]['score']
    scores = {"test_score":test_score,"final_dev_score":final_dev_score,"tradeoff":tradeoff}
    return scores

In [8]:
score_list = []
for i in range(1,19):
    new_df_list,name_list = get_new_df(i)
    scores = get_scores(new_df_list,name_list)
    scores['index'] = i
    score_list.append(scores)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
scores

{'test_score': 1.2900014486376508,
 'final_dev_score': 1.2397273316949056,
 'tradeoff': 0.22,
 'index': 18}

In [10]:
df_score = pd.DataFrame(score_list)

In [11]:
df_score['std'] = df_score[['test_score','final_dev_score']].std(axis=1)
df_score['avg'] = df_score[['test_score','final_dev_score']].mean(axis=1)
df_score['avg-3std'] = df_score['avg']-3*df_score['std']

In [12]:
df_score.sort_values(['std','avg'])

,test_score,final_dev_score,tradeoff,index,std,avg,avg-3std
1,1.265729,1.259205,0.26,2,0.004613,1.262467,1.248629
2,1.278559,1.263680,0.23,3,0.010521,1.271120,1.239557
6,1.291807,1.268147,0.24,7,0.016730,1.279977,1.229786
0,1.269692,1.244168,0.30,1,0.018048,1.256930,1.202786
3,1.288924,1.261695,0.25,4,0.019254,1.275309,1.217547
4,1.285281,1.252504,0.24,5,0.023177,1.268893,1.199362
7,1.294217,1.260278,0.24,8,0.023999,1.277247,1.205251
9,1.284704,1.248693,0.23,10,0.025464,1.266698,1.190307
5,1.282424,1.236027,0.22,6,0.032808,1.259225,1.160802
16,1.303284,1.254725,0.23,17,0.034336,1.279004,1.175996


In [ ]:
new_df_list,name_list = get_new_df(2)

In [13]:
df_final_test = new_df_list[3]
df_final_test['merge_mean'] = df_final_test[name_list].mean(axis=1)

In [14]:
tradeoff = 0.26

In [15]:
df_final_test['Label'] = (df_final_test['merge_mean']>tradeoff).apply(int)
df_final_test['ID'] = df_final_test['Id']

In [16]:
keep_cols = ['ID','Dialogue_id','Utterance_id','Label']
week_name = "final2_0923"

In [17]:
df_final_test[keep_cols].to_csv(f'data/TAL-JNU_任务1.csv',index=False)

In [18]:
df_final_test['Label'].mean()

0.14434740417273167

In [19]:
df_final_test.shape

(4122, 34)

In [20]:
for df in new_df_list:
    print(df['Label'].mean())

0.16424751718869365
0.1614487415592388
0.12913553895410887
0.14434740417273167
